# Hyperparameter Tuning for DistilBERT (30% Subset)

In [ ]:
# Install required packages
%pip install --quiet --upgrade optuna transformers>=4.3.0

## 1. Imports and Configuration

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import torch
import optuna

from sklearn.metrics import accuracy_score, f1_score
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
)
from sklearn.model_selection import train_test_split

# Reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# Paths
SPLITS_DIR = "C:/Users/indur/OneDrive - University of Westminster/GitHub/FYP_Project/Models/Ai_Genuine_Reviews/FinalDataSet"
HPO_OUTPUT_DIR = "C:/Users/indur/OneDrive - University of Westminster/GitHub/FYP_Project/Models/Ai_Genuine_Reviews/OutPuts"
os.makedirs(HPO_OUTPUT_DIR, exist_ok=True)

# HPO settings
MAX_LENGTH = 256
N_TRIALS = 5
PATIENCE = 3

## 2. Load & Sample Data

In [ ]:
# Load full splits
train_df = pd.read_csv(os.path.join(SPLITS_DIR, "train.csv"))
val_df = pd.read_csv(os.path.join(SPLITS_DIR, "val.csv"))

# Stratified 30% sampling
train_df, _ = train_test_split(
    train_df,
    train_size=0.3,
    stratify=train_df.label,
    random_state=SEED
)
val_df, _ = train_test_split(
    val_df,
    train_size=0.3,
    stratify=val_df.label,
    random_state=SEED
)

# Reset indices
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

print(f"Using {len(train_df)} train / {len(val_df)} val (stratified 30%)")

## 3. Tokenizer

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

## 4. Dataset Definition

In [ ]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer):
        enc = tokenizer(
            texts.tolist(),
            padding="max_length",
            truncation=True,
            max_length=MAX_LENGTH,
            return_tensors="pt",
        )
        self.input_ids = enc.input_ids
        self.attention_mask = enc.attention_mask
        self.labels = torch.tensor(labels.values, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx],
        }

train_ds = ReviewDataset(train_df.clean_review, train_df.label, tokenizer)
val_ds = ReviewDataset(val_df.clean_review, val_df.label, tokenizer)

## 5. Metrics

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds),
    }

## 6. Trainer Base Arguments

In [ ]:
base_args = TrainingArguments(
    output_dir=HPO_OUTPUT_DIR,
    do_train=True,
    do_eval=True,
    eval_strategy="epoch",
    save_strategy="no",            
    per_device_eval_batch_size=32,
    fp16=torch.cuda.is_available(),
    num_train_epochs = 10,
    seed=SEED,
    report_to="none",
    metric_for_best_model="f1",    
    greater_is_better=True,        
    load_best_model_at_end=False,  
)

## 7. Model Initialization

In [ ]:
def model_init():
    return DistilBertForSequenceClassification.from_pretrained(
        "distilbert-base-uncased",
        num_labels=2
    )

## 8. Hyperparameter Search Space

In [ ]:
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.3), 
        "warmup_steps": trial.suggest_int("warmup_steps", 0, 500),     
    }

## 9. Initialize Trainer

In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=base_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=PATIENCE)],
)

## 10. Run Hyperparameter Search

In [ ]:
best_run = trainer.hyperparameter_search(
    hp_space=hp_space,
    direction="maximize",
    backend="optuna",
    n_trials=N_TRIALS,
    compute_objective=lambda metrics: metrics["eval_f1"],
)

## 11. Save Best Parameters

In [ ]:
print("Best hyperparameters:", best_run.hyperparameters)
with open(os.path.join(HPO_OUTPUT_DIR, "best_params.json"), "w") as f:
    json.dump(best_run.hyperparameters, f, indent=2)